Converting all of the code in the python file to a Jupyter notebook.

As I am coming back to it I see some problems that need solving. 

The goal of this script:
 - Take a number from user input
 - take user's number and create that many random numbers with range
    > range needs to be reverse engineered manually from finding the min and max of HT articles on apple
 - exclude the numbers that are already present in csv file
    > possible way to do this: https://stackoverflow.com/questions/42999093/generate-random-number-in-range-excluding-some-numbers
 - with this list of numbers, create links
 - scrape these links for article name, description, and last modified date
 - add this content to csv file

In [1]:
import requests
import pandas as pd
import html_file  # for testing
from bs4 import BeautifulSoup as bs
from time import sleep
from random import randint
from datetime import date

This class takes an int and adds it to 200000 (which is the min article number)

In [21]:
class Webpage:

    def __init__(self,num):
        self.num = num
        self.url_main = 'https://support.apple.com/en-us/HT'
        self.url_num_base = 200000
        self.headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-us',
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15'}
        self.url_complete = str(self.url_main + str(self.url_num_base + self.num))

    def get_html(self):
        try:
            r = requests.get(self.url_complete, headers=self.headers)
            if r.status_code == 200:
                html = r.text
                r.close
                return html
            else:
                html = self.url_complete
                r.close
                return html
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)

    def create_soup(self):
        html = self.get_html()
        if html == self.url_complete:
            soup = self.url_complete
        else:
            soup = bs(html, features="html.parser")
        return soup

    def get_info(self):
        """Gets url, title, description, and other information"""
        soup = self.create_soup()
        try:
            info_url = soup.head.find(attrs={"rel": "canonical"})['href']
            info_title = soup.title.text
            info_description = soup.find(attrs={"name": "description"})['content'].replace('\n','')
            info_last_modified_date = soup.find(attrs={"class": "mod-date"}).time['datetime']

            doc_info = [info_url, info_title, info_description, info_last_modified_date]

        except AttributeError as e:
            doc_info = [soup, 'Non 200 Status Code']
        return doc_info


In [23]:
testing = Webpage(3122)
print(testing.url_complete)

print(testing.get_info())

https://support.apple.com/en-us/HT203122
['https://support.apple.com/en-us/HT203122', 'If you see the Restore screen on your iPhone, iPad, or iPod touch - Apple Support', 'If you see the\xa0Restore screen on\xa0your iPhone, iPad, or iPod touch, learn what to do.', '2021-09-23']


This test looks to have some funny escape characters in it.  will have to clean this up, but later!